<a href="https://colab.research.google.com/github/TacticalCoders/NLP/blob/main/Transformers_for_NLP/ch01/Multi_Head_Attention_Sub_Layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from scipy.special import softmax

In [3]:
print("1단계 : 4차원(512->4)의 3개의 입력(Input : 3 inputs, d_model=4)")
x = np.array([[1.0, 0.0, 1.0, 0.0],
              [0.0, 2.0, 0.0, 2.0],
              [1.0, 1.0, 1.0, 1.0]])
print(x)

1단계 : Input : 3 inputs, d_model=4
[[1. 0. 1. 0.]
 [0. 2. 0. 2.]
 [1. 1. 1. 1.]]


In [5]:
print("2단계: 4행 3(64->3)열의 가중치 행렬(d_model=4)")
print("w_query")
w_query = np.array([[1, 0, 1],
                    [1, 0, 0],
                    [0, 0, 1],
                    [0, 1, 1]])
print(w_query)

2단계: 4행 3열의 가중치 행렬(d_model=4)
w_query
[[1 0 1]
 [1 0 0]
 [0 0 1]
 [0 1 1]]


In [6]:
print("w_key")
w_key =np.array([[0, 0, 1],
                 [1, 1, 0],
                 [0, 1, 0],
                 [1, 1, 0]])
print(w_key)

w_key
[[0 0 1]
 [1 1 0]
 [0 1 0]
 [1 1 0]]


In [7]:
print("w_value")
w_value = np.array([[0, 2, 0],
                    [0, 3, 0],
                    [1, 0, 3],
                    [1, 1, 0]])
print(w_value)

w_value
[[0 2 0]
 [0 3 0]
 [1 0 3]
 [1 1 0]]


참고로 이 가중치 행렬들은 훈련으로 얻어진다. 

그리고 오리지널 트랜스포머에서 사용하는 가중치 행렬은 모두  512행, 64열이다.

In [8]:
print("3단계: Q, K, V 벡터를 얻기 위한 행렬곱")

print("Queries: x * w_query")
Q=np.matmul(x,w_query)
print(Q)

3단계: Q, K, V 벡터를 얻기 위한 행렬곱
Queries: x * w_query
[[1. 0. 2.]
 [2. 2. 2.]
 [2. 1. 3.]]


In [9]:
print("Keys: x * w_key")
K=np.matmul(x,w_key)
print(K)

Keys: x * w_key
[[0. 1. 1.]
 [4. 4. 0.]
 [2. 3. 1.]]


In [10]:
print("Values: x * w_value")
V=np.matmul(x,w_value)
print(V)

Values: x * w_value
[[1. 2. 3.]
 [2. 8. 0.]
 [2. 6. 3.]]


In [11]:
print("4단계: 스케일된 내적 어텐션(스코어)")
k_d=1 #루트 dk, 즉 루트 3(1.73)을 1로 
attention_scores = (Q @ K.transpose())/k_d
print(attention_scores)

4단계: 스케일된 내적 어텐션(스코어)
[[ 2.  4.  4.]
 [ 4. 16. 12.]
 [ 4. 12. 10.]]


첫 번째 단어 x1에 대한 각 입력에 대한 어텐션 스코어는 2, 4, 4이다.



In [12]:
print("5단계: 각 벡터에 대한 스케일된 softmax 어텐션 점수")
attention_scores[0] = softmax(attention_scores[0])
attention_scores[1] = softmax(attention_scores[1])
attention_scores[2] = softmax(attention_scores[2])
print(attention_scores[0])
print(attention_scores[1])
print(attention_scores[2])

5단계: 각 벡터에 대한 스케일된 softmax 어텐션 점수
[0.06337894 0.46831053 0.46831053]
[6.03366485e-06 9.82007865e-01 1.79861014e-02]
[2.95387223e-04 8.80536902e-01 1.19167711e-01]


In [13]:
print("6단계: 최종 어텐션 표현(소프트맥스를 통해 얻은 어텐션 가중치와 V곱하기)")
print(V[0])
print(V[1])
print(V[2])
print("Attention 1")
attention1=attention_scores[0].reshape(-1,1)
attention1=attention_scores[0][0]*V[0]
print(attention1)

print("Attention 2")
attention2=attention_scores[0][1]*V[1]
print(attention2)

print("Attention 3")
attention3=attention_scores[0][2]*V[2]
print(attention3)

6단계: 최종 어텐션 표현(소프트맥스를 통해 얻은 어텐션 가중치와 V곱하기)
[1. 2. 3.]
[2. 8. 0.]
[2. 6. 3.]
Attention 1
[0.06337894 0.12675788 0.19013681]
Attention 2
[0.93662106 3.74648425 0.        ]
Attention 3
[0.93662106 2.80986319 1.40493159]


각 Attention1,2,3은 첫 번째 단어 x1에 대한 3개의 어텐션 값(3개의 key에 대한)을 의미한다.

In [14]:
print("7단계: 어텐션 결과 합산")
attention_input1=attention1+attention2+attention3
print(attention_input1)

7단계: 어텐션 결과 합산
[1.93662106 6.68310531 1.59506841]


이 벡터가 어텐션 출력 행렬의 첫 번째 행이 될 것이다.

In [15]:
print("8단계: 나머지 입력들에 대해 단계 1~7수행")
# d_model=64로 학습된 가중치에 의한 3개의 어텐션 값이 있다고 가정한다.
# 오리지널 차원들이 서브레이어의 출력애 도달했을 때의 모양을 확인하고자 한다.

attention_head1=np.random.random((3, 64))
print(attention_head1)

8단계: 나머지 입력들에 대해 단계 1~7수행
[[0.63208227 0.88215193 0.46111181 0.22761256 0.27002387 0.12910001
  0.52562444 0.87578265 0.0150151  0.87130635 0.85992993 0.20953625
  0.45722595 0.9990534  0.97924333 0.72237541 0.13746608 0.14045825
  0.47802267 0.44887708 0.87250681 0.17429645 0.63953612 0.36261146
  0.95102285 0.5803502  0.38272882 0.12280673 0.06709225 0.08832493
  0.77419665 0.90520022 0.01862232 0.16388754 0.19748884 0.73842788
  0.17325862 0.90400657 0.43980883 0.12596864 0.7364913  0.90660408
  0.55270865 0.48566135 0.43114592 0.08358064 0.403377   0.69637234
  0.42704368 0.23811052 0.87309943 0.78767508 0.22635059 0.72231367
  0.40684142 0.75949704 0.80211791 0.15881727 0.15468519 0.51845551
  0.73208059 0.85086951 0.28730033 0.81989916]
 [0.05223882 0.08944417 0.74096282 0.47165554 0.08865405 0.7996842
  0.54437672 0.38972261 0.72489333 0.6033972  0.30139281 0.92369921
  0.27788959 0.33992954 0.81745664 0.648442   0.69401929 0.7042959
  0.89075788 0.75137118 0.0729366  0.2017328 

즉, 3개의 단어에 대한 최종 어텐션 값을 산출한 것이다. 첫 번째 벡터는 첫 번째 key 단어와의 어텐션 값인 것이다.

In [16]:
print("9단계: 8개 헤드들의 출력")
z0h1=np.random.random((3, 64))
z1h2=np.random.random((3, 64))
z2h3=np.random.random((3, 64))
z3h4=np.random.random((3, 64))
z4h5=np.random.random((3, 64))
z5h6=np.random.random((3, 64))
z6h7=np.random.random((3, 64))
z7h8=np.random.random((3, 64))
print("shape of one head",z0h1.shape,"dimension of 8 heads",64*8)

9단계: 8개 헤드들의 출력
shape of one head (3, 64) dimension of 8 heads 512


In [17]:
print("10단계: 8개의 헤드 연결하기")
output_attention=np.hstack((z0h1,z1h2,z2h3,z3h4,z4h5,z5h6,z6h7,z7h8))
print(output_attention)

10단계: 8개의 헤드 연결하기
[[0.16985392 0.84172047 0.16652259 ... 0.13520991 0.8416682  0.23594613]
 [0.31855329 0.55018356 0.25944598 ... 0.34304078 0.50181853 0.06545083]
 [0.23471275 0.38045653 0.86151236 ... 0.10429432 0.11677889 0.96405482]]


최종 어텐션 출력은 입력과 똑같은 크기 x*512의 행렬이다.

그리고 이 최종 표현은 훈련된 가중치 행렬로 곱해진다. 